In [1]:
from keras.models import Sequential

In [2]:
from keras.layers import Dense, Activation

In [3]:
model = Sequential([
    Dense(32, input_shape=(10,),activation='relu'),
    Dense(2, activation='softmax')
])

Instructions for updating:
Colocations handled automatically by placer.


note: 32 is the number of nodes/neurons in first layer. 10 is the total number of input varables

In [7]:
model.layers

#### PRACTICE

In [8]:
from keras.datasets import mnist

MNIST dataset: contains 70000 28x28 grayscale images with 10 different classes. Keras splits it in a training set with 60000 instances and a testing set with 10000 instances.

In [9]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [13]:
x_train.shape

(60000, 28, 28)

In [14]:
x_test.shape

(10000, 28, 28)

In [15]:
y_train.shape

(60000,)

In [16]:
y_test.shape

(10000,)

In [17]:
type(x_train)

numpy.ndarray

To feed the images to a convolutional neural network we transform the dataframe to four dimensions. 

In [19]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [21]:
x_train /= 255
x_test /= 255

In [27]:
x_train.shape[0]

60000

In [28]:
x_train = x_train.reshape(x_train.shape[0],28,28,1)
x_test = x_test.reshape(x_test.shape[0],28,28,1)

In [29]:
x_test.shape

(10000, 28, 28, 1)

In [30]:
x_train.shape

(60000, 28, 28, 1)

In [36]:
x_test[0][0].shape

(28, 1)

transform our labels into a one-hot encoding using the to_categorical method from Keras

In [43]:
from keras.utils.np_utils import to_categorical

In [44]:
y_train = to_categorical(y_train,10)   #10 classes
y_test = to_categorical(y_test,10)

In [47]:
y_train.shape

(60000, 10)

The easiest way of creating a model in Keras is by using the sequential API, which lets you stack one layer after the other. The problem with the sequential API is that it doesn’t allow models to have multiple inputs or outputs, which are needed for some problems.

To create a convolutional neural network we only need to create a Sequential object and use the add function to add layers.

In [48]:
from keras.models import Sequential

In [49]:
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout 

In [50]:
model = Sequential()

In [56]:
model.add(Conv2D(nb_filter=32,nb_row=5,nb_col=5, activation='relu', input_shape=x_train.shape[1:]))

In [55]:
x_train.shape[1:]

(28, 28, 1)

In [58]:
model.add(Conv2D(nb_filter=32,nb_row=5,nb_col=5))

In [60]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [65]:
model.add(Dropout(p=0.25))

In [66]:
model.add(Conv2D(nb_filter=64,nb_row=3,nb_col=3, activation='relu'))
model.add(Conv2D(nb_filter=64,nb_row=3,nb_col=3, activation='relu'))

In [67]:
model.layers

In [68]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [69]:
model.add(Dropout(p=0.25))

In [70]:
model.layers

In [71]:
model.add(Flatten())

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [72]:
model.output_shape

(None, 576)

In [73]:
model.input_shape

(None, 28, 28, 1)

In [74]:
model.layers

In [75]:
model.add(Dense(256, activation='relu'))

In [76]:
model.layers

In [77]:
model.add(Dropout(p=0.25))

In [78]:
model.layers

In [79]:
model.add(Dense(10, activation='softmax'))

In [80]:
model.layers

In [81]:
len(model.layers)

15

The code above first of creates a Sequential object and adds a few convolutional, maxpooling and dropout layers.

It then flattens the output and passes it two a last dense and dropout layer before passing it to our output layer.

using Sequential API

In [82]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

model = Sequential([
    Conv2D(nb_filter=32, nb_row=5, nb_col=5, activation='relu', input_shape=x_train.shape[1:]),
    Conv2D(nb_filter=32, nb_row=5, nb_col=5, activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(p=0.25),
    Conv2D(nb_filter=64, nb_row=3, nb_col=3, activation='relu'),
    Conv2D(nb_filter=64, nb_row=3, nb_col=3, activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(p=0.25),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(p=0.25),
    Dense(output_dim=10, activation='softmax')
])

In [83]:
model.layers

In [84]:
len(model.layers)

12

using Funtional API

In [85]:
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Input

inputs = Input(shape=x_train.shape[1:])

inputToNextLayer = Conv2D(nb_filter=32, nb_row=5, nb_col=5, activation='relu')(inputs)
#  nb_filter: Number of convolution filters to use.
inputToNextLayer = Conv2D(nb_filter=32, nb_row=5, nb_col=5, activation='relu')(inputToNextLayer)

inputToNextLayer = MaxPooling2D(pool_size=(2,2))(inputToNextLayer)

inputToNextLayer = Dropout(p=0.25)(inputToNextLayer)

inputToNextLayer = Conv2D(nb_filter=64, nb_row=3, nb_col=3, activation='relu')(inputToNextLayer)
inputToNextLayer = Conv2D(nb_filter=64, nb_row=3, nb_col=3, activation='relu')(inputToNextLayer)

inputToNextLayer = MaxPooling2D(pool_size=(2,2))(inputToNextLayer)

inputToNextLayer = Dropout(p=0.25)(inputToNextLayer)

inputToNextLayer = Flatten()(inputToNextLayer)

inputToNextLayer = Dense(output_dim=256, activation='relu')(inputToNextLayer)

inputToNextLayer = Dropout(p=0.25)(inputToNextLayer)

prediction = Dense(output_dim=10, activation='softmax')(inputToNextLayer)

model = Model(input=inputs, output=prediction)

In [86]:
model.layers

In [87]:
len(model.layers)

13

The loss function is a measure on how good our model is at achieving the given objective.

An optimizer is used to minimize the loss(objective) function by updating the weights using the gradients.

In [90]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [91]:
from keras.preprocessing.image import ImageDataGenerator

In [92]:
datagen = ImageDataGenerator(rotation_range=10,
                            zoom_range=0.1,
                            width_shift_range=0.1,
                            height_shift_range=0.1)

In [96]:
epoch = 2
batch_size = 32

model.fit_generator(datagen.flow(x_train,y_train,batch_size=batch_size),nb_epoch=epoch, validation_data=(x_test,y_test),
                   samples_per_epoch=x_train.shape[0]//batch_size)

Instructions for updating:
Use tf.cast instead.
Epoch 1/2
1856/1875 [============================>.] - ETA: 0s - loss: 1.9952 - acc: 0.2829

C:\Users\nirmal.silwal\AppData\Local\Continuum\anaconda3\envs\learningmate\lib\site-packages\keras\engine\training.py:1573: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


1888/1875 [==============================] - 15s - loss: 1.9852 - acc: 0.2865 - val_loss: 0.8985 - val_acc: 0.7412
Epoch 2/2
1888/1875 [==============================] - 13s - loss: 1.1298 - acc: 0.6271 - val_loss: 0.3116 - val_acc: 0.9098


In [97]:
epoch = 3
batch_size = 32

model.fit_generator(datagen.flow(x_train,y_train,batch_size=batch_size),nb_epoch=epoch, validation_data=(x_test,y_test),
                   samples_per_epoch=x_train.shape[0]//batch_size)

Epoch 1/3
1888/1875 [==============================] - 12s - loss: 0.7520 - acc: 0.7564 - val_loss: 0.2103 - val_acc: 0.9431
Epoch 2/3
1888/1875 [==============================] - 13s - loss: 0.5332 - acc: 0.8400 - val_loss: 0.1480 - val_acc: 0.9554
Epoch 3/3
1888/1875 [==============================] - 13s - loss: 0.4654 - acc: 0.8623 - val_loss: 0.1482 - val_acc: 0.9599


In [98]:
epoch = 3
batch_size = 32

model.fit_generator(datagen.flow(x_train,y_train,batch_size=batch_size),nb_epoch=epoch, validation_data=(x_test,y_test),
                   samples_per_epoch=x_train.shape[0]//batch_size)

Epoch 1/3
1888/1875 [==============================] - 13s - loss: 0.3599 - acc: 0.8845 - val_loss: 0.0965 - val_acc: 0.9684
Epoch 2/3
1888/1875 [==============================] - 12s - loss: 0.3552 - acc: 0.8888 - val_loss: 0.1017 - val_acc: 0.9683
Epoch 3/3
1888/1875 [==============================] - 13s - loss: 0.3288 - acc: 0.9010 - val_loss: 0.0733 - val_acc: 0.9766


In [99]:
epoch = 10
batch_size = 32

model.fit_generator(datagen.flow(x_train,y_train,batch_size=batch_size),nb_epoch=epoch, validation_data=(x_test,y_test),
                   samples_per_epoch=x_train.shape[0]//batch_size)

Epoch 1/10
1888/1875 [==============================] - 13s - loss: 0.3031 - acc: 0.9094 - val_loss: 0.0923 - val_acc: 0.9721
Epoch 2/10
1888/1875 [==============================] - 12s - loss: 0.2478 - acc: 0.9206 - val_loss: 0.0790 - val_acc: 0.9764
Epoch 3/10
1888/1875 [==============================] - 13s - loss: 0.2200 - acc: 0.9354 - val_loss: 0.0649 - val_acc: 0.9805
Epoch 4/10
1888/1875 [==============================] - 12s - loss: 0.2596 - acc: 0.9211 - val_loss: 0.0728 - val_acc: 0.9773
Epoch 5/10
1888/1875 [==============================] - 13s - loss: 0.2311 - acc: 0.9290 - val_loss: 0.0745 - val_acc: 0.9766
Epoch 6/10
1888/1875 [==============================] - 13s - loss: 0.2032 - acc: 0.9401 - val_loss: 0.0592 - val_acc: 0.9809
Epoch 7/10
1888/1875 [==============================] - 13s - loss: 0.2065 - acc: 0.9333 - val_loss: 0.0658 - val_acc: 0.9793
Epoch 8/10
1888/1875 [==============================] - 12s - loss: 0.2265 - acc: 0.9317 - val_loss: 0.0560 - val_acc:

In [100]:
epoch = 5
batch_size = 32

model.fit_generator(datagen.flow(x_train,y_train,batch_size=batch_size),nb_epoch=epoch, validation_data=(x_test,y_test),
                   samples_per_epoch=x_train.shape[0]//batch_size)

Epoch 1/5
1888/1875 [==============================] - 13s - loss: 0.1598 - acc: 0.9470 - val_loss: 0.0397 - val_acc: 0.9865
Epoch 2/5
1888/1875 [==============================] - 13s - loss: 0.1480 - acc: 0.9492 - val_loss: 0.0545 - val_acc: 0.9827
Epoch 3/5
1888/1875 [==============================] - 13s - loss: 0.1896 - acc: 0.9370 - val_loss: 0.0468 - val_acc: 0.9852
Epoch 4/5
1888/1875 [==============================] - 13s - loss: 0.1307 - acc: 0.9645 - val_loss: 0.0654 - val_acc: 0.9790
Epoch 5/5
1888/1875 [==============================] - 14s - loss: 0.1474 - acc: 0.9502 - val_loss: 0.0664 - val_acc: 0.9798
